In [1]:
from utility import scraper
from bs4 import BeautifulSoup as soup
from tqdm.notebook import tqdm

import pandas as pd
import numpy as np

import re
import string

from utility.s3_file_transfer import upload_df_to_s3

In [2]:
preprocessed_data_dir = './preprocessed_data'
processed_data_dir = './processed_data'

## 1. Scrape Tier List Table (UGG)
- Updated daily to weekly

### 1.1 Extract Raw Data

In [3]:
def extract_from_tier_list_row(row,divison):
    
    # extract information from the row
    divison = divison.capitalize() 
    lane = row.find('img',{'class':'tier-list-role'})['alt'].capitalize() 
    champion = row.find('strong',{'class':'champion-name'}).text
    try:
        regex = re.compile('.*rt-td tier.*')
        tier = row.find('div',{'class':regex}).find('span').text
    except:
        tier = float('nan')
    try:
        regex = re.compile('.*rt-td winrate.*')
        win_rate = row.find('div',{'class':regex}).find('span').text
    except:
        win_rate = float('nan')
    try:
        regex = re.compile('.*rt-td pickrate.*')
        pick_rate = row.find('div',{'class':regex}).find('span').text
    except:
        pick_rate = float('nan')
    try:
        regex = re.compile('.*rt-td banrate.*')
        ban_rate = row.find('div',{'class':regex}).find('span').text
    except:
        ban_rate = float('nan')
    try:
        counter_raw_list = row.find('div',{'class':'against-container'}).findAll('div',{'class':'against'})
        counter_list = list()
        for counter in counter_raw_list:
            counter_champ = counter.find('a')['href'].split('/')[3].capitalize() 
            counter_list.append(counter_champ)
        counters = ','.join(counter_list)
    except:
        counters = float('nan')
    try:
        regex = re.compile('.*rt-td matches.*')
        matches = row.find('div',{'class':regex}).find('span').text
    except:
        matches = float('nan')
    # store all into a dictionary
    row_dict = {'divison':divison,'lane':lane,'champion':champion,
                'tier':tier,'win_rate':win_rate,'pick_rate':pick_rate,
                'ban_rate':ban_rate,'counters':counters,'matches':matches}
    
    return row_dict


def generate_divison_tier_table(champ_tier_list_soup,divison):
    
    tier_list_table = champ_tier_list_soup.find('div',{'class':'tier-list-page'})
    champ_rows = tier_list_table.findAll('div',{'class':'rt-tr-group'})
    
    division_dict = dict()
    for i,row in enumerate(tqdm(champ_rows)):
        division_dict[i] = extract_from_tier_list_row(row,divison)
    df_division = pd.DataFrame.from_dict(division_dict, orient='index').sort_values(by=['win_rate'],ascending=False)
    return df_division

def scrape_ugg_tier_list():
    
    divisions = ['iron','bronze','silver','gold','platinum','diamond','master','grandmaster','challenger']
    df_tier_list = pd.DataFrame()

    for divsion in tqdm(divisions):
        # generate the table for the given division
        url = f'https://u.gg/lol/tier-list?rank={divsion}'
        champ_tier_list_soup = scraper.get_page_selenium(url)
        df_divsion = generate_divison_tier_table(champ_tier_list_soup,divison=divsion)
        # append to parent table
        df_tier_list = pd.concat([df_tier_list,df_divsion],ignore_index=True)
    return df_tier_list

In [4]:
df_tier_list = scrape_ugg_tier_list()

  0%|          | 0/9 [00:00<?, ?it/s]

  0%|          | 0/223 [00:00<?, ?it/s]

  0%|          | 0/223 [00:00<?, ?it/s]

  0%|          | 0/219 [00:00<?, ?it/s]

  0%|          | 0/211 [00:00<?, ?it/s]

  0%|          | 0/211 [00:00<?, ?it/s]

  0%|          | 0/209 [00:00<?, ?it/s]

  0%|          | 0/205 [00:00<?, ?it/s]

  0%|          | 0/202 [00:00<?, ?it/s]

  0%|          | 0/197 [00:00<?, ?it/s]

In [5]:
df_tier_list.to_csv('./temp/ugg_tier_list_raw.csv',index=False)

### 1.2 Preprocess
- Correct names in `counters`

In [6]:
def adjust_name(name):
    adjusted_name = name.translate(str.maketrans('', '', string.punctuation))
    adjusted_name = adjusted_name.replace(' ','').lower().capitalize()
    return adjusted_name

def correct_counters(counters,correction_champ_names,name_correction_dict):
    for name in correction_champ_names:
        if name in counters:
            counters = counters.replace(name,name_correction_dict[name])
    return counters

In [7]:
champ_names = pd.Series(df_tier_list['champion'].unique())
corrected_champ_names = champ_names[champ_names.apply(lambda x: ' ' in x or "'" in x)]
correction_champ_names = pd.Series(corrected_champ_names).apply(adjust_name)

name_correction_dict = dict(zip(correction_champ_names,corrected_champ_names))

In [8]:
df_tier_list['counters'] = df_tier_list['counters'].apply(lambda x: correct_counters(x,correction_champ_names,name_correction_dict))

In [9]:
df_tier_list.to_csv(f'{preprocessed_data_dir}/ugg_tier_list.csv',index=False)

### 1.3 Process and Upload

In [15]:
def convert_to_float(percent):
    if isinstance(percent,str):
        return round(float(percent.replace('%',''))/100,4)
    else:
        return float('nan')

In [16]:
df_tier_list = pd.read_csv(f'{preprocessed_data_dir}/ugg_tier_list.csv')
df_champ_roles = pd.read_csv(f'{preprocessed_data_dir}/wiki_champ_roles.csv')
df_champ_details = pd.read_csv(f'{preprocessed_data_dir}/wiki_champion_details.csv')

In [17]:
tier_list_set = set(df_tier_list['champion'].unique())
champ_roles_set = set(df_champ_roles['champion'].unique())
champ_details_set = set(df_champ_details['champion'].unique())

In [18]:
if tier_list_set == champ_roles_set and champ_roles_set == champ_details_set:
    # turn matches into int dtype
    df_tier_list['matches'] = df_tier_list['matches'].apply(lambda x: int(x.replace(',','')))
    
    # convert rates to float dtype
    df_tier_list['win_rate'] = df_tier_list['win_rate'].apply(convert_to_float)
    df_tier_list['pick_rate'] = df_tier_list['pick_rate'].apply(convert_to_float)
    df_tier_list['ban_rate'] = df_tier_list['ban_rate'].apply(convert_to_float)
    
    # map tiers into int dtype
    tier_mapping = {'S+':0,'S':1,'A':2,'B':3,'C':4,'D':5}
    df_tier_list['tier'] = df_tier_list['tier'].map(tier_mapping)
    
    df_champ_stats = df_tier_list
    df_champ_stats.rename(columns={'divison':'rank'},inplace=True)

In [20]:
df_champ_stats.head()

,rank,lane,champion,tier,win_rate,pick_rate,ban_rate,counters,matches
0,Iron,Jungle,Skarner,1,0.6019,0.008,0.0,"Nidalee,Amumu,Cho'Gath,Xin Zhao,Nocturne,Sejua...",422
1,Iron,Jungle,Rammus,0,0.5711,0.041,0.0,"Elise,Nunu,Lillia,Wukong,Amumu,Sejuani,Warwick",2180
2,Iron,Mid,Mordekaiser,2,0.5679,0.007,0.0,"Tryndamere,Pyke,Azir,Galio,Leblanc,Morgana,Swain",361
3,Iron,Mid,Teemo,1,0.5593,0.010,0.0,"Vladimir,Anivia,Xerath,Karthus,Tristana,Ezreal...",540
4,Iron,Top,Singed,1,0.5554,0.011,0.0,"Tahm Kench,Shen,Yone,Heimerdinger,Gangplank,Dr...",596


In [21]:
# save to local
df_champ_stats.to_csv(f'{processed_data_dir}/champion_stats.csv',index=False)
# upload to s3
# upload_df_to_s3(df_champ_stats,'peter-ff15-data/champion_stats.csv')